# 최종 점검

### 1. ReLU 함수 사용할 때 주로 쓰는 가중치 초깃값은??

$$\sqrt{2 \over n}\$$
(He initialization)

### 2. 이진 분류 문제를 모델링할 때, 출력층에서 무슨 함수를 쓰는 것이 적합할까?

시그모이드 함수

### 3. Optimizer 예를 3가지 들어라

GD(Gradient Descent): 기울기 이용<br>
Momentum: exponentially weighted average 이용, <br>
AdaGrad: learning rate decay 이용, 학습률을 조정

### 4. One-Hot encoding이란 무엇인가?

선택지의 개수의 차원을 가지는 벡터에서 원하는 선택지의 원소를 1, 나머지 선택지의 원소는 0이 되도록 하는 표현 방법

### 5. Opencv로 이미지 데이터들을 numpy array로 만들었다. 이 데이터들을 모두 같은 크기 (640x800)으로 만들기 위해 필요한 함수는?

cv2.resize(name, dsize, interpolation)
* numpy.reshape의 경우 전체 크기 변경이 불가함
* numpy.resize의 경우 크기가 변경될 경우 크기를 강제로 변경함 (의도치 않은 원소 반복 발생 가능)
* cv2.resize의 경우 interpolation 설정 가능함

In [12]:
# Import libraries for 5)
import os
import cv2
import numpy as np

In [ ]:
# Test code
base = '/Users/seolyumin/Downloads/cats/'
for file_name in os.listdir(base):
    path = base + file_name
    if '.jpg' in path:
        im = cv2.imread(path, 0)
        im_res = cv2.resize(im, (640, 800))
        print("original size:", im.shape, "changed size:", im_res.shape)

### 6. Tensorflow로 CNN모델을 설계하고 코드를 설명할 것(조건 이외에는 자유)

조건 1) class로 구현<br>
조건 2) batch normalization 구현<br>
조건 3) Xavier 초깃값 구현<br>
조건 4) 최소 3개 이상의 layer 구현<br>

In [13]:
import numpy as np
import tensorflow as tf

In [15]:
# MNIST 숫자 손글씨 데이터셋을 가지고 여러 모델을 구현한 후 가장 정확도가 높았던 모델을 선정
class CNN(tf.keras.Model):
    def __init__(self):
        super(CNN, self).__init__()
        # 3 convolutional layers, and fully connected layer at the end
        # Activation: ReLU(with He), Sigmoid(with Xavier)
        self.conv1 = tf.keras.layers.Conv2D(filters=32, kernel_size=5, strides=1, padding='same', kernel_initializer='he_normal', activation='relu')
        self.pool1 = tf.keras.layers.MaxPool2D(padding='same')
        self.conv2 = tf.keras.layers.Conv2D(filters=64, kernel_size=5, strides=1, padding='same', kernel_initializer='he_normal', activation='relu')
        self.pool2 = tf.keras.layers.MaxPool2D(padding='same')
        self.conv3 = tf.keras.layers.Conv2D(filters=128, kernel_size=3, strides=1, padding='same', kernel_initializer='he_normal', activation='relu')
        self.pool3 = tf.keras.layers.MaxPool2D(padding='same')
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(units=256, kernel_initializer='he_normal', activation='relu')
        self.dropout = tf.keras.layers.Dropout(rate=0.4)
        self.dense2 = tf.keras.layers.Dense(units=10, kernel_initializer='glorot_normal', activation='sigmoid') # Xavier normalization
        self.BN = tf.keras.layers.BatchNormalization()
    def call(self, input_data, training=False):
        # Convolutional layer 1 - Convolution(32, 3) -> ReLU(He) -> Batch Norm. -> Pooling
        res = self.conv1(input_data)
        res = self.BN(res)
        res = self.pool1(res)
        # Convolutional layer 2 - Convolution(64, 3) -> ReLU(He) -> Batch Norm. -> Pooling
        res = self.conv2(res)
        res = self.BN(res)
        res = self.pool2(res)
        # Convolutional layer 3 - Convolution(128, 3) -> ReLU(He) -> Batch Norm. -> Pooling
        res = self.conv3(res)
        res = self.BN(res)
        res = self.pool3(res)
        # Fully connected layer part
        res = self.flatten(res)
        res = self.dense1(res)
        res = self.BN(res)
        res = self.dropout(res)
        res = self.dense2(res)
        return res

In [16]:
# MNIST 데이터셋 불러오기
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
train_images = np.reshape(train_images, (60000, 28, 28, 1))
train_images = train_images.astype(np.float32) / 255. # 0과 1 사이의 값으로 변환

In [19]:
test_images = np.reshape(test_images, (10000, 28, 28, 1))
test_images = test_images.astype(np.float32) / 255. # 0과 1 사이의 값으로 변환

In [ ]:
# Shape 변경 확인
train_images[0]

In [18]:
# 약식 모델 train
model5 = tf.keras.Sequential([
    # Convolutional layer 1 - Convolution(32, 3) -> ReLU(He) -> Batch Norm. -> Pooling
    tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding='same', kernel_initializer='he_normal', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(padding='same'),
    # Convolutional layer 2 - Convolution(64, 3) -> ReLU(He) -> Batch Norm. -> Pooling
    tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding='same', kernel_initializer='he_normal', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(padding='same'),
    # Convolutional layer 3 - Convolution(128, 3) -> ReLU(He) -> Batch Norm. -> Pooling
    tf.keras.layers.Conv2D(filters=128, kernel_size=3, padding='same', kernel_initializer='he_normal', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(padding='same'),
    # Fully connected layer part - Flatten -> Dense(256) -> ReLU(He) -> Batch Norm. -> Dropout(0.4) -> Dense(10)
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=256, kernel_initializer='he_normal', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(rate=0.4),
    tf.keras.layers.Dense(units=10, kernel_initializer='glorot_normal', activation='sigmoid')
])
model5.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model5.fit(train_images, train_labels, epochs=10, validation_split=0.2)
model5.summary()

Epoch 1/10
1500/1500 [==============================] - 44s 29ms/step - loss: 0.1165 - accuracy: 0.9639 - val_loss: 0.0415 - val_accuracy: 0.9872
Epoch 2/10
1500/1500 [==============================] - 47s 31ms/step - loss: 0.0508 - accuracy: 0.9840 - val_loss: 0.0439 - val_accuracy: 0.9872
Epoch 3/10
1500/1500 [==============================] - 48s 32ms/step - loss: 0.0368 - accuracy: 0.9880 - val_loss: 0.0338 - val_accuracy: 0.9897
Epoch 4/10
1500/1500 [==============================] - 52s 35ms/step - loss: 0.0312 - accuracy: 0.9904 - val_loss: 0.0357 - val_accuracy: 0.9899
Epoch 5/10
1500/1500 [==============================] - 51s 34ms/step - loss: 0.0274 - accuracy: 0.9910 - val_loss: 0.0385 - val_accuracy: 0.9893
Epoch 6/10
1500/1500 [==============================] - 50s 33ms/step - loss: 0.0219 - accuracy: 0.9932 - val_loss: 0.0425 - val_accuracy: 0.9882
Epoch 7/10
1500/1500 [==============================] - 49s 33ms/step - loss: 0.0176 - accuracy: 0.9945 - val_loss: 0.0290 -

In [20]:
# 약식 모델 test
model5.evaluate(test_images, test_labels)

313/313 [==============================] - 3s 8ms/step - loss: 0.0279 - accuracy: 0.9928


[0.027936438098549843, 0.9927999973297119]